# Final project: The battle of Neighborhoods

## PART 1: Creating the dataframe

In [1]:
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_cities_by_international_visitors'
table = pd.read_html(url)
df = pd.DataFrame(table[0])
pd.set_option('display.max_columns', 5000)
df.columns = ['Rank_arrivals','Rank_income','City','Country','Arrivals_2018','Arrivals_income_2016','Growthin arrivals','Income_billion$']
df = df[0:]
df.shape

(140, 8)

In [2]:
df.head()

,Rank_arrivals,Rank_income,City,Country,Arrivals_2018,Arrivals_income_2016,Growthin arrivals,Income_billion$
0,1.0,11.0,Hong Kong,Hong Kong,29262700.0,8370000.0,5.0%,6.84
1,2.0,1.0,Bangkok,Thailand,24177500.0,21470000.0,7.7%,14.84
2,3.0,2.0,London,United Kingdom,19233000.0,19880000.0,−3.0%,19.76
3,4.0,NaN,Macau,Macau,18931400.0,NaN,9.2%,NaN
4,5.0,6.0,Singapore,Singapore,18551200.0,12110000.0,5.3%,12.54


Because all cases of missing values were happened at least three columns, I decide to delete these rows. 

In [3]:
df_na = df.dropna(axis=0, how='any')
df_drop = df_na.drop(['Rank_arrivals','Rank_income','Arrivals_income_2016','Growthin arrivals'],axis=1)
df_sort = df_drop.sort_values(by=['Income_billion$'],ascending=0)

In [4]:
df_new = pd.DataFrame([])
df_new['City'] = df_sort['City'].values
df_new['Country'] = df_sort['Country'].values
df_new['Arrivals_2018'] = df_sort['Arrivals_2018'].values
df_new['Income_billion$'] = df_sort['Income_billion$'].values
df_new.shape

(60, 4)

In [ ]:
df_new.head(10)

,City,Country,Arrivals_2018,Income_billion$
0,Dubai,United Arab Emirates,15920700.0,31.30
1,London,United Kingdom,19233000.0,19.76
2,New York City,United States,13600000.0,18.52
3,Bangkok,Thailand,24177500.0,14.84
4,Tokyo,Japan,9985100.0,13.48
5,Paris,France,17560200.0,12.88
6,Singapore,Singapore,18551200.0,12.54
7,Kuala Lumpur,Malaysia,13434300.0,11.34
8,Seoul,South Korea,8431400.0,11.30
9,Taipei,Taiwan,9597800.0,9.60


**I just chose top 10 cities, which have the best income**

In [ ]:
Ranking = pd.DataFrame([1,2,3,4,5,6,7,8,9,10])
top_10 = df_new.head(10)
top_10.insert(0,'Ranking',Ranking)

top_10.head(10)

,Ranking,City,Country,Arrivals_2018,Income_billion$
0,1,Dubai,United Arab Emirates,15920700.0,31.30
1,2,London,United Kingdom,19233000.0,19.76
2,3,New York City,United States,13600000.0,18.52
3,4,Bangkok,Thailand,24177500.0,14.84
4,5,Tokyo,Japan,9985100.0,13.48
5,6,Paris,France,17560200.0,12.88
6,7,Singapore,Singapore,18551200.0,12.54
7,8,Kuala Lumpur,Malaysia,13434300.0,11.34
8,9,Seoul,South Korea,8431400.0,11.30
9,10,Taipei,Taiwan,9597800.0,9.60


**Searching coordinates of each citites**

In [ ]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [ ]:
top_10["Location"]= top_10["City"].str.cat(top_10['Country'], sep =",")
top_10

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Ranking,City,Country,Arrivals_2018,Income_billion$,Location
0,1,Dubai,United Arab Emirates,15920700.0,31.30,"Dubai,United Arab Emirates"
1,2,London,United Kingdom,19233000.0,19.76,"London,United Kingdom"
2,3,New York City,United States,13600000.0,18.52,"New York City,United States"
3,4,Bangkok,Thailand,24177500.0,14.84,"Bangkok,Thailand"
4,5,Tokyo,Japan,9985100.0,13.48,"Tokyo,Japan"
5,6,Paris,France,17560200.0,12.88,"Paris,France"
6,7,Singapore,Singapore,18551200.0,12.54,"Singapore,Singapore"
7,8,Kuala Lumpur,Malaysia,13434300.0,11.34,"Kuala Lumpur,Malaysia"
8,9,Seoul,South Korea,8431400.0,11.30,"Seoul,South Korea"
9,10,Taipei,Taiwan,9597800.0,9.60,"Taipei,Taiwan"


In [ ]:
geolocator = Nominatim(user_agent="thaitran")
Latitude = []
Longitude = []
for i in range(0,10):
    address = top_10.Location[i]
    location = geolocator.geocode(address)
    print(address,'has latitude: ',location.latitude,', and longtitude: ',location.longitude)
    Latitude.insert(i,location.latitude)
    Longitude.insert(i,location.longitude)

top_10['Latitude'] = Latitude
top_10['Longitude'] = Longitude
top_10.head(10)

Dubai,United Arab Emirates has latitude:  25.0657 , and longtitude:  55.1713
London,United Kingdom has latitude:  51.5073219 , and longtitude:  -0.1276474
New York City,United States has latitude:  40.7127281 , and longtitude:  -74.0060152
Bangkok,Thailand has latitude:  13.7542529 , and longtitude:  100.493087
Tokyo,Japan has latitude:  35.6828387 , and longtitude:  139.7594549
Paris,France has latitude:  48.8566969 , and longtitude:  2.3514616
Singapore,Singapore has latitude:  1.357107 , and longtitude:  103.8194992


**Mapping these locations in world's map**

In [ ]:
map_top_10 = top_10.drop(['Arrivals_2018','Location'],axis=1)
map_top_10.Country[2] = 'United States of America' #Appropriate name by following the indicator of map
map_top_10.head(10)

In [ ]:
!wget --quiet https://raw.githubusercontent.com/parulnith/Visualising-Geospatial-data-with-Python/master/world-countries.json -O world-countries.json
country_geo = r'world-countries.json'
world_map = folium.Map(location=[20,0],zoom_start=2)
for i in range(0,len(top_10)):
    folium.Marker([map_top_10.iloc[i]['Latitude'],map_top_10.iloc[i]['Longitude']],popup=map_top_10.iloc[i]['City']).add_to(world_map)
world_map.choropleth(
    geo_data=country_geo,
    data=map_top_10,
    columns=['Country','Income_billion$'],
    key_on='feature.properties.name',
    fill_color='YlGnBu', 
    fill_opacity=0.8, 
    line_opacity=0.3,
    legend_name='Income (MasterCard) from international visitors - billion US dollars'
)
world_map

## PART 2: Determining the advantage points by clustering all venues with its geograpic data.

**1. Define Foursquare Credentials and its version**

In [ ]:
CLIENT_ID = 'UWJJQEM3PRAXT0G0OEEJF0KEQYLVLC1RQ4LCINHXDSIMVLEH'
CLIENT_SECRET = 'MSWWUQODJHCLCIX3C0MDB5CBV0Q1QY0IDM4POE1UJAZSTFJJ'
VERSION='20180412'

**2. Selecting all venues within the cities **

In [ ]:
import requests
def getNearbyVenues(names, latitudes, longitudes):
    LIMIT=1000
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION,
            lat, 
            lng,  
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue',
                  'Venue ID',           
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
city_venues = getNearbyVenues(names=top_10['City'],
                                   latitudes=top_10['Latitude'],
                                   longitudes=top_10['Longitude'])

In [ ]:
city_venues.head(10)

* Note: If you can retrieve all data, which is related to Vistits Count, Checkin Count, liked count, and rating score, you can know better the business oppotunities in these areas.

def getInfoVenues(Venue_IDs):
    venues_list=[]
    for i in range(0,len(Venue_IDs)):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            Venue_IDs[i], 
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION)
        print (url)
        # make the GET request
        visitsCount = requests.get(url).json()["response"]['venue']['stats']['visitsCount']
        checkinsCount = requests.get(url).json()["response"]['venue']['stats']['checkinsCount']
        likes = requests.get(url).json()["response"]['venue']['likes']['count']
        rating = requests.get(url).json()["response"]['venue']['rating']
        print (likes)
        # return only relevant information for each nearby venue
        venues_list.append(
            Venue_IDs[i],
            visitsCount,
            checkinsCount,
            likes,
            rating)

    info_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    info_venues.columns = ['Venue_ID',
                  'Visits Count',
                  'Checkins Count', 
                  'Likes Count',
                  'Rating']
    
    return(info_venues)

info_venues = getInfoVenues(Venue_IDs = Dubai_data['Venue ID'])
info_venues.head(50)

**3. Clustering venues in each city by these coordinates**

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

### The Investigation in Dubai

In [ ]:
Dubai_d =city_venues[city_venues['City'].str.contains("Dubai")]
Dubai_data = Dubai_d.reset_index(drop=True)
k_data=Dubai_data.loc[:,['Venue ID','Venue Latitude','Venue Longitude']]
k_data.head(10)

In [ ]:
K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = k_data[['Venue Latitude']]
X_axis = k_data[['Venue Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(k_data[k_data.columns[1:3]]) # Compute k-means clustering.
k_data['Cluster_label'] = kmeans.fit_predict(k_data[k_data.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(k_data[k_data.columns[1:3]]) # Labels of each point
centroids = pd.DataFrame(centers)
centroids.columns = ['Adv Latitude','Adv Longitude']
Cluster = pd.DataFrame(['D1','D2','D3'])
City = pd.DataFrame(['Dubai','Dubai','Dubai'])
Country = pd.DataFrame(['United Arab Emirates','United Arab Emirates','United Arab Emirates'])
centroids.insert(0,'Advantage points(Adv)',Cluster)
centroids.insert(0,'City',City)
centroids.insert(0,'Country',Country)
Dubai_rpt = centroids.copy()
Dubai_rpt.head()

### The investigation in London

In [ ]:
London_d=(city_venues[city_venues['City'].str.contains("London")])
London_data = London_d.reset_index(drop=True)
k_data=London_data.loc[:,['Venue ID','Venue Latitude','Venue Longitude']]
k_data.head()

In [ ]:
K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = k_data[['Venue Latitude']]
X_axis = k_data[['Venue Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(k_data[k_data.columns[1:3]]) # Compute k-means clustering.
k_data['Cluster_label'] = kmeans.fit_predict(k_data[k_data.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(k_data[k_data.columns[1:3]]) # Labels of each point
centroids = pd.DataFrame(centers)
centroids.columns = ['Adv Latitude','Adv Longitude']
Cluster = pd.DataFrame(['L1','L2','L3'])
City = pd.DataFrame(['London','London','London'])
Country = pd.DataFrame(['United Kingdom','United Kingdom','United Kingdom'])
centroids.insert(0,'Advantage points(Adv)',Cluster)
centroids.insert(0,'City',City)
centroids.insert(0,'Country',Country)
London_rpt = centroids.copy()
London_rpt.head()

### The investigation in New York(NY)

In [ ]:
NY_d=(city_venues[city_venues['City'].str.contains("New York City")])
NY_data = NY_d.reset_index(drop=True)
k_data=NY_data.loc[:,['Venue ID','Venue Latitude','Venue Longitude']]
k_data.head()

In [ ]:
K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = k_data[['Venue Latitude']]
X_axis = k_data[['Venue Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 2, init ='k-means++')
kmeans.fit(k_data[k_data.columns[1:3]]) # Compute k-means clustering.
k_data['Cluster_label'] = kmeans.fit_predict(k_data[k_data.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(k_data[k_data.columns[1:3]]) # Labels of each point
centroids = pd.DataFrame(centers)
centroids.columns = ['Adv Latitude','Adv Longitude']
Cluster = pd.DataFrame(['NY1','NY2','NY3'])
City = pd.DataFrame(['New York City','New York City','New York City'])
Country = pd.DataFrame(['United States of America','United States of America','United States of America'])
centroids.insert(0,'Advantage points(Adv)',Cluster)
centroids.insert(0,'City',City)
centroids.insert(0,'Country',Country)
NY_rpt = centroids.copy()
NY_rpt.head()

### The investigation in Bangkok

In [ ]:
Bangkok_d=(city_venues[city_venues['City'].str.contains("Bangkok")])
Bangkok_data = Bangkok_d.reset_index(drop=True)
k_data=Bangkok_data.loc[:,['Venue ID','Venue Latitude','Venue Longitude']]
k_data.head()

In [ ]:
K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = k_data[['Venue Latitude']]
X_axis = k_data[['Venue Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(k_data[k_data.columns[1:3]]) # Compute k-means clustering.
k_data['Cluster_label'] = kmeans.fit_predict(k_data[k_data.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(k_data[k_data.columns[1:3]]) # Labels of each point
centroids = pd.DataFrame(centers)
centroids.columns = ['Adv Latitude','Adv Longitude']
Cluster = pd.DataFrame(['L1','L2','L3'])
City = pd.DataFrame(['Bangkok','Bangkok','Bangkok'])
Country = pd.DataFrame(['Thailand','Thailand','Thailand'])
centroids.insert(0,'Advantage points(Adv)',Cluster)
centroids.insert(0,'City',City)
centroids.insert(0,'Country',Country)
Bangkok_rpt = centroids.copy()
Bangkok_rpt.head()

### The investigation in Tokyo(Tk)

In [ ]:
Tokyo_d=(city_venues[city_venues['City'].str.contains("Tokyo")])
Tokyo_data = Tokyo_d.reset_index(drop=True)
k_data=Tokyo_data.loc[:,['Venue ID','Venue Latitude','Venue Longitude']]
k_data.head()

In [ ]:
K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = k_data[['Venue Latitude']]
X_axis = k_data[['Venue Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(k_data[k_data.columns[1:3]]) # Compute k-means clustering.
k_data['Cluster_label'] = kmeans.fit_predict(k_data[k_data.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(k_data[k_data.columns[1:3]]) # Labels of each point
centroids = pd.DataFrame(centers)
centroids.columns = ['Adv Latitude','Adv Longitude']
Cluster = pd.DataFrame(['Tk1','Tk2','Tk3'])
City = pd.DataFrame(['Tokyo','Tokyo','Tokyo'])
Country = pd.DataFrame(['Japan','Japan','Japan'])
centroids.insert(0,'Advantage points(Adv)',Cluster)
centroids.insert(0,'City',City)
centroids.insert(0,'Country',Country)
Tokyo_rpt = centroids.copy()
Tokyo_rpt.head()

### The investigation in Paris(P)

In [ ]:
Paris_d=(city_venues[city_venues['City'].str.contains("Paris")])
Paris_data = Paris_d.reset_index(drop=True)
k_data=Paris_data.loc[:,['Venue ID','Venue Latitude','Venue Longitude']]
k_data.head()

In [ ]:
K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = k_data[['Venue Latitude']]
X_axis = k_data[['Venue Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(k_data[k_data.columns[1:3]]) # Compute k-means clustering.
k_data['Cluster_label'] = kmeans.fit_predict(k_data[k_data.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(k_data[k_data.columns[1:3]]) # Labels of each point
centroids = pd.DataFrame(centers)
centroids.columns = ['Adv Latitude','Adv Longitude']
Cluster = pd.DataFrame(['P1','P2','P3'])
City = pd.DataFrame(['Paris','Paris','Paris'])
Country = pd.DataFrame(['France','France','France'])
centroids.insert(0,'Advantage points(Adv)',Cluster)
centroids.insert(0,'City',City)
centroids.insert(0,'Country',Country)
Paris_rpt = centroids.copy()
Paris_rpt.head()

### The investigation in Singapore(Sp)

In [ ]:
Sp_d=(city_venues[city_venues['City'].str.contains("Singapore")])
Sp_data = Sp_d.reset_index(drop=True)
k_data=Sp_data.loc[:,['Venue ID','Venue Latitude','Venue Longitude']]
k_data.head()

In [ ]:
K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = k_data[['Venue Latitude']]
X_axis = k_data[['Venue Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(k_data[k_data.columns[1:3]]) # Compute k-means clustering.
k_data['Cluster_label'] = kmeans.fit_predict(k_data[k_data.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(k_data[k_data.columns[1:3]]) # Labels of each point
centroids = pd.DataFrame(centers)
centroids.columns = ['Adv Latitude','Adv Longitude']
Cluster = pd.DataFrame(['Sp1','Sp2','Sp3'])
City = pd.DataFrame(['Singapore','Singapore','Singapore'])
Country = pd.DataFrame(['Singapore','Singapore','Singapore'])
centroids.insert(0,'Advantage points(Adv)',Cluster)
centroids.insert(0,'City',City)
centroids.insert(0,'Country',Country)
Sp_rpt = centroids.copy()
Sp_rpt.head()

### The investigation in Kuala Lumpur(Kl)

In [ ]:
Kl_d = (city_venues[city_venues['City'].str.contains("Kuala Lumpur")])
Kl_data = Kl_d.reset_index(drop=True)
k_data = Kl_data.loc[:,['Venue ID','Venue Latitude','Venue Longitude']]
k_data.head()

In [ ]:
K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = k_data[['Venue Latitude']]
X_axis = k_data[['Venue Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(k_data[k_data.columns[1:3]]) # Compute k-means clustering.
k_data['Cluster_label'] = kmeans.fit_predict(k_data[k_data.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(k_data[k_data.columns[1:3]]) # Labels of each point
centroids = pd.DataFrame(centers)
centroids.columns = ['Adv Latitude','Adv Longitude']
Cluster = pd.DataFrame(['Kl1','Kl2','Kl3'])
City = pd.DataFrame(['Kuala Lumpur','Kuala Lumpur','Kuala Lumpur'])
Country = pd.DataFrame(['Malaysia','Malaysia','Malaysia'])
centroids.insert(0,'Advantage points(Adv)',Cluster)
centroids.insert(0,'City',City)
centroids.insert(0,'Country',Country)
Kl_rpt = centroids.copy()
Kl_rpt.head()

### The investigation in Seoul(Se)

In [ ]:
Se_d = (city_venues[city_venues['City'].str.contains("Seoul")])
Se_data = Se_d.reset_index(drop=True)
k_data = Se_data.loc[:,['Venue ID','Venue Latitude','Venue Longitude']]
k_data.head()

In [ ]:
K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = k_data[['Venue Latitude']]
X_axis = k_data[['Venue Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(k_data[k_data.columns[1:3]]) # Compute k-means clustering.
k_data['Cluster_label'] = kmeans.fit_predict(k_data[k_data.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(k_data[k_data.columns[1:3]]) # Labels of each point
centroids = pd.DataFrame(centers)
centroids.columns = ['Adv Latitude','Adv Longitude']
Cluster = pd.DataFrame(['Se1','Se2','Se3'])
City = pd.DataFrame(['Seoul','Seoul','Seoul'])
Country = pd.DataFrame(['South Korea','South Korea','South Korea'])
centroids.insert(0,'Advantage points(Adv)',Cluster)
centroids.insert(0,'City',City)
centroids.insert(0,'Country',Country)
Se_rpt = centroids.copy()
Se_rpt.head()

### The investigation in Tapei(Tp)

In [ ]:
Tp_d = (city_venues[city_venues['City'].str.contains("Taipei")])
Tp_data = Tp_d.reset_index(drop=True)
k_data = Tp_data.loc[:,['Venue ID','Venue Latitude','Venue Longitude']]
k_data.head()

In [ ]:
K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = k_data[['Venue Latitude']]
X_axis = k_data[['Venue Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 2, init ='k-means++')
kmeans.fit(k_data[k_data.columns[1:3]]) # Compute k-means clustering.
k_data['Cluster_label'] = kmeans.fit_predict(k_data[k_data.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(k_data[k_data.columns[1:3]]) # Labels of each point
centroids = pd.DataFrame(centers)
centroids.columns = ['Adv Latitude','Adv Longitude']
Cluster = pd.DataFrame(['Tp1','Tp2','Tp3'])
City = pd.DataFrame(['Tapei','Tapei','Tapei'])
Country = pd.DataFrame(['Taiwan','Taiwan','Taiwan'])
centroids.insert(0,'Advantage points(Adv)',Cluster)
centroids.insert(0,'City',City)
centroids.insert(0,'Country',Country)
Tp_rpt = centroids.copy()
Tp_rpt.head()

### Assembly all dataframes of those countries into one

In [ ]:
All_rpt = pd.concat([Dubai_rpt,London_rpt,NY_rpt,Bangkok_rpt,Tokyo_rpt,Paris_rpt,Sp_rpt,Kl_rpt,Se_rpt,Tp_rpt], ignore_index=True)
All_rpt.columns = ['Country','City','Advantage points','Adv Latitude','Adv Longitude']
All_rpt.head(50)

## PART 3: Exploring and clustering the familiar bussiness models in different avantage points of top 10 countries which have best income from international visitors

**1. Selecting all venues around the clusters**

In [ ]:
import requests
def getNearbyVenues(names, latitudes, longitudes):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION,
            lat, 
            lng,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Advantage points', 
                  'Adv Latitude', 
                  'Adv Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
adv_venues = getNearbyVenues(names=All_rpt['Advantage points'],
                                   latitudes=All_rpt['Adv Latitude'],
                                   longitudes=All_rpt['Adv Longitude'])

**2. Listing venues of the geographic clusters**

In [ ]:
adv_venues.head(1000)

 **3. Counting all venues with the same clusters**

In [ ]:
adv_venues.groupby('Advantage points').count()

**4. Mean dataframe of each venue in each Neighborhood**

In [ ]:
# one hot encoding
adv_onehot = pd.get_dummies(adv_venues[['Venue Category']], prefix="", prefix_sep="")
 # insert neighborhood column to the first column
adv_onehot.insert(loc=0, column='Advantage points', value=adv_venues['Advantage points'])
adv_grouped = adv_onehot.groupby('Advantage points').mean().reset_index()
adv_grouped.head()

**5. Ranking common venues in each geographic cluster**

In [ ]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Advantage points']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
adv_venues_sorted = pd.DataFrame(columns=columns)
adv_venues_sorted['Advantage points'] = adv_grouped['Advantage points']

for ind in np.arange(adv_grouped.shape[0]):
    adv_venues_sorted.iloc[ind, 1:] = return_most_common_venues(adv_grouped.iloc[ind, :], num_top_venues)

adv_venues_sorted.head()

**6. Clustering similar groups of venue categories for the business models of each cluster**

In [ ]:
adv_grouped_clustering = adv_grouped.drop('Advantage points', 1)
K_clusters = range(1,15)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
score = [kmeans[i].fit(adv_grouped_clustering).score(adv_grouped_clustering) for i in range(len(kmeans))]# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
kclusters = 9 # Number of clusters
adv_grouped_clustering = adv_grouped.drop('Advantage points', 1)
# k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(adv_grouped_clustering)
# add labels
adv_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)
adv_merged = All_rpt
# merge toronto_grouped with toronto_data
adv_merged = adv_merged.join(adv_venues_sorted.set_index('Advantage points'), on='Advantage points')
adv_merged.head()

In [ ]:
adv_merged_mod = adv_merged.copy()
for i in range(0,len(adv_merged_mod['Cluster_Labels'])):
    adv_merged_mod.Cluster_Labels[i] += 1
adv_all_rpt = adv_merged_mod.copy()
adv_all_rpt

**7. Showing these clusters with comparison of their bussiness models on global map and local maps**

In [ ]:
import numpy as np
map_clusters = folium.Map(location=[20,0], zoom_start=2)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(adv_all_rpt['Adv Latitude'],adv_all_rpt['Adv Longitude'],adv_all_rpt['Advantage points'],adv_all_rpt['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
map_clusters

In [ ]:
latitude = map_top_10.Latitude[0]
longitude = map_top_10.Longitude[0]
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=14)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(adv_all_rpt['Adv Latitude'],adv_all_rpt['Adv Longitude'],adv_all_rpt['Advantage points'],adv_all_rpt['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
map_clusters

In [ ]:
latitude = map_top_10.Latitude[1]
longitude = map_top_10.Longitude[1]
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=16)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(adv_all_rpt['Adv Latitude'],adv_all_rpt['Adv Longitude'],adv_all_rpt['Advantage points'],adv_all_rpt['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
map_clusters

In [ ]:
latitude = map_top_10.Latitude[2]
longitude = map_top_10.Longitude[2]
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=16)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(adv_all_rpt['Adv Latitude'],adv_all_rpt['Adv Longitude'],adv_all_rpt['Advantage points'],adv_all_rpt['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
map_clusters

In [ ]:
latitude = map_top_10.Latitude[3]
longitude = map_top_10.Longitude[3]
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=16)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(adv_all_rpt['Adv Latitude'],adv_all_rpt['Adv Longitude'],adv_all_rpt['Advantage points'],adv_all_rpt['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
map_clusters

In [ ]:
latitude = map_top_10.Latitude[4]
longitude = map_top_10.Longitude[4]
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=16)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(adv_all_rpt['Adv Latitude'],adv_all_rpt['Adv Longitude'],adv_all_rpt['Advantage points'],adv_all_rpt['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
map_clusters

In [ ]:
latitude = map_top_10.Latitude[5]
longitude = map_top_10.Longitude[5]
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=16)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(adv_all_rpt['Adv Latitude'],adv_all_rpt['Adv Longitude'],adv_all_rpt['Advantage points'],adv_all_rpt['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
map_clusters

In [ ]:
latitude = map_top_10.Latitude[6]
longitude = map_top_10.Longitude[6]
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=16)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(adv_all_rpt['Adv Latitude'],adv_all_rpt['Adv Longitude'],adv_all_rpt['Advantage points'],adv_all_rpt['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
map_clusters

In [ ]:
latitude = map_top_10.Latitude[7]
longitude = map_top_10.Longitude[7]
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=16)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(adv_all_rpt['Adv Latitude'],adv_all_rpt['Adv Longitude'],adv_all_rpt['Advantage points'],adv_all_rpt['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
map_clusters

In [ ]:
latitude = map_top_10.Latitude[8]
longitude = map_top_10.Longitude[8]
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=16)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(adv_all_rpt['Adv Latitude'],adv_all_rpt['Adv Longitude'],adv_all_rpt['Advantage points'],adv_all_rpt['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
map_clusters

In [ ]:
latitude = map_top_10.Latitude[9]
longitude = map_top_10.Longitude[9]
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=16)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(adv_all_rpt['Adv Latitude'],adv_all_rpt['Adv Longitude'],adv_all_rpt['Advantage points'],adv_all_rpt['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
map_clusters